In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import json
import pandas as pd
import ast
from tqdm import tqdm
from src.models.llm import call
from src.models.utils import *
from src.models.config import get_model_config
from src.translator.translator import Translator

# from prompts import *
trans = Translator()

[nltk_data] Downloading package punkt to /home/iouzzani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/iouzzani/anaconda3/envs/myenv/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
# model = 'llama3_8b'
# def llm_config_func(llm):
#     llm.temperature = 0
#     llm.max_tokens = 4096
#     return llm
# config = get_model_config(model)

### llama2 7b
# model = 'llama2_7b'
# def llm_config_func(llm):
#     llm.temperature = 0
#     llm.max_tokens = 2048
#     return llm
# config = get_model_config(model)

### gpt3.5
model = 'gpt-3.5-turbo'
def llm_config_func(llm):
    llm.temperature = 0.8
    llm.max_tokens = 4096
    return llm
config = get_model_config(model)

In [6]:
import os

mc_file_paths = []
for filename in os.listdir("src/data_model_cards/claude3"):
    mc_file_paths.append("src/data_model_cards/claude3/"+filename)

f = open(mc_file_paths[0], "r")
mc_text = f.read()

In [7]:
# extracting scientific jargon from English model cards

prompt = [
    """You are a language expert with a focus on machine translation. Your task is to analyze the following text and identify scientific jargon or terminology within the field of machine learning that might be challenging for machine translation software to accurately translate. List these terms in a clear, concise format.
        Response Format: [Term 1, Term 2, Term 3, ...]""",
        
    """The following text is:
       "{text}"
    """.format(text=mc_text)
]

res = call(
    prompt,
    llm_config_func,
    has_system_prompt=True,
    model_version=model,
    verbose=True,
    **config
)

term_lst = list(map(lambda s: s.strip(), res[1:-1].split(","))) # could be better 

print(res, term_lst)


1
org-q4KwOHTjWKGwZDHrQlsOPcp3
You are a language expert with a focus on machine translation. Your task is to analyze the following text and identify scientific jargon or terminology within the field of machine learning that might be challenging for machine translation software to accurately translate. List these terms in a clear, concise format.
        Response Format: [Term 1, Term 2, Term 3, ...]
The following text is:
       "# Model Card for nvidia/segformer-b1-finetuned-cityscapes-1024-1024

The model nvidia/segformer-b1-finetuned-cityscapes-1024-1024 is a semantic segmentation model based on the SegFormer architecture, which consists of a hierarchical Transformer encoder and a lightweight All-MLP decoder. It has been fine-tuned on the Cityscapes dataset at a resolution of 1024x1024 pixels to segment driving scenes into 19 categories.

## Model Details

### Model Description

Model Architecture:
- SegFormer consists of a hierarchical Transformer encoder to generate multi-scale f

In [ ]:
# if terms not in scientific jargon dictionary, add to dictionary with translations

scientific_jargon_dict = trans.read_json_file('src/scientific_jargon_dict.json')
    
tmp_dict = {"Chinese":{},"Russian":{}, "Japanese":{}, "French":{}, "Arabic":{}}
for term in term_lst:
    if term not in scientific_jargon_dict["Chinese"]:
        tmp_dict["Chinese"][term] = ""
        tmp_dict["Russian"][term] = ""
        tmp_dict["Japanese"][term] = ""
        tmp_dict["French"][term] = ""
        tmp_dict["Arabic"][term] = ""

print("Translating using Seamless M4T ...")

trans.translate_term_m4t(tmp_dict, "English", "Chinese")
print("Chinese translation complete")
    
trans.translate_term_m4t(tmp_dict, "English", "Russian")
print("Russian translation complete")
    
trans.translate_term_m4t(tmp_dict, "English", "Japanese")
print("Japanese translation complete")
    
trans.translate_term_m4t(tmp_dict, "English", "French")
print("French translation complete")
    
trans.translate_term_m4t(tmp_dict, "English", "Arabic")
print("Arabic translation complete")

    
scientific_jargon_dict.update(tmp_dict)
trans.write_file_to_json(scientific_jargon_dict, 'src/scientific_jargon_dict.json')

Translating using Seamless M4T ...
Chinese translation complete
Russian translation complete
